In [1]:
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
import pandas
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# CARREGANDO DADOS

# importando dataset
dataset = fetch_ucirepo(id=109) # 178 registros e 13 atributos
  
# coletando as informações
data_frame = dataset.data.original

In [3]:
# TRATANDO DADOS

print("PRÉ TRATAMENTO: ", len(data_frame))

# removendo colunas com muitos nulos
tolerancia = len(data_frame) * 0.7
data_frame = data_frame.dropna(axis=1, thresh=tolerancia)

# removendo duplicados
print("VALORES DUPLICADOS: ", data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()

# convertendo colunas categóricas em valores inteiros
conversores = {}
for coluna in data_frame.columns:
    if (data_frame[coluna].dtype == type(object)):
        conversor = LabelEncoder()
        data_frame[coluna] = conversor.fit_transform(data_frame[coluna])
        conversores[coluna] = conversor

print("PÓS TRATAMENTO: ", len(data_frame))

PRÉ TRATAMENTO:  178
VALORES DUPLICADOS:  0
PÓS TRATAMENTO:  178


In [4]:
print(data_frame.columns)

Index(['Alcohol', 'Malicacid', 'Ash', 'Alcalinity_of_ash', 'Magnesium',
       'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols',
       'Proanthocyanins', 'Color_intensity', 'Hue',
       '0D280_0D315_of_diluted_wines', 'Proline', 'class'],
      dtype='object')


In [5]:
# DIVIDINDO O DATASET TRATADO

atributos = data_frame.drop(["class"], axis=1)
respostas = data_frame[["class"]]

a_treino, a_teste, r_treino, r_teste = train_test_split(atributos, respostas, test_size=0.4, random_state=42)


In [6]:
# CRIANDO PARÂMETROS PARA OS MODELOS

lista_parametros = [
    {"id": "1", "criterion": "gini", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "2", "criterion": "gini", "max_depth": 10, "min_samples_split": 10, "min_samples_leaf": 5},
    {"id": "3", "criterion": "entropy", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "5", "criterion": "entropy", "max_depth": 10, "min_samples_split": 10, "min_samples_leaf": 5},
]

In [7]:
# APLICANDO MODELO

# criando a lista de resultados finais
lista_resultados = []

for parametros in lista_parametros:
    # criando regressor
    regressor = tree.DecisionTreeClassifier(
        criterion=parametros["criterion"],
        max_depth=parametros["max_depth"],
        min_samples_split=parametros["min_samples_split"],
        min_samples_leaf=parametros["min_samples_leaf"],
    )

    # treinando o modelo
    regressor.fit(a_treino, r_treino)

    # prevendo respostas
    r_previsao = regressor.predict(a_teste)

    # calculando métricas
    acuracia = accuracy_score(r_teste, r_previsao)
    f1 = f1_score(r_teste, r_previsao, average="weighted")

    # salvando resultados
    lista_resultados.append([parametros["id"], acuracia, f1])

In [8]:
# EXIBINDO RESULTADOS
tabela_resultados = pandas.DataFrame(lista_resultados, columns=["Identificador", "Acurácia", "F1-score"])
print(tabela_resultados)

  Identificador  Acurácia  F1-score
0             1  0.930556  0.930996
1             2  0.930556  0.930996
2             3  0.944444  0.944650
3             5  0.902778  0.902263
